# Virtual Large Language Transformer Micro Server
---


##  Setup all lib and env:
**Python Lib:**
>* FastApi - *for client access* 
>* redis - *for base data*
>* websockets - *for desktop control* 
>* aiortc - *for desktop streaming*

```
pip install fastapi
pip install uvicorn
pip install transformers
pip install torch
pip install redis
pip install aioredis
pip install python-socketio[client]

```

In [1]:
# Setup all lib and env

# fastapi :
try:
    import fastapi                      
    print('fastapi: already installed')
except ImportError:
  !python -m pip install -q fastapi
  print('Installed fastapi')
    
# uvicorn :
try:
    import uvicorn                      
    print('uvicorn: already installed')
except ImportError:
  !python -m pip install -q uvicorn
  print('Installed uvicorn')
    
# transformers :
try:
    import transformers                      
    print('transformers: already installed')
except ImportError:
  !python -m pip install -q transformers
  print('Installed transformers')
    
# torch :
try:
    import torch                      
    print('torch: already installed')
except ImportError:
  !python -m pip install -q torch
  print('Installed torch')
    
# redis :
try:
    import redis                      
    print('redis: already installed')
except ImportError:
  !python -m pip install -q redis
  print('Installed redis')
    
# aioredis :
try:
    import aioredis                      
    print('aioredis: already installed')
except ImportError:
  !python -m pip install -q aioredis
  print('Installed aioredis')
    
# python-socketio[client] :
try:
    import socketio                      
    print('python-socketio[client]: already installed')
except ImportError:
  !python -m pip install -q python-socketio[client]
  print('Installed socketio')


fastapi: already installed
uvicorn: already installed
transformers: already installed
torch: already installed
redis: already installed
aioredis: already installed
python-socketio[client]: already installed


---
## Setup Language Trasnformer Stream API:

## Import libraries:

In [2]:
import json
import asyncio
from fastapi import FastAPI, WebSocket
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from pydantic import BaseModel
import redis
import aioredis
import socketio


---
## Define the FastAPI app, Redis configuration, and Socket.IO client:

In [3]:
app = FastAPI()

# Redis configuration
REDIS_URL = "redis://localhost:6379"
redis_client = redis.Redis.from_url(REDIS_URL)
redis_pubsub = redis_client.pubsub()

# Socket.IO client
sio = socketio.AsyncClient()

---
## Load the pre-trained model and tokenizer:
**Choose a pre-trained model:**

Select a suitable pre-trained model from the Hugging Face Model Hub [Hugginface/models](https://huggingface.co/models). 
> For example, you can use "gpt2" or "gpt2-medium" if you want a model similar to GPT-3.


## Prepare your dataset:

> Transform your dataset into a format suitable for training. You'll need to tokenize your text data and create input sequences. You can use the GPT2Tokenizer for this purpose.

In [4]:
# Load GPT-2 model and tokenizer
model_name = "gpt2-large"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

---

## Fine-tune the model:

>Fine-tune the pre-trained model on your dataset using a suitable training loop. You can use the Trainer class from the

---

---

## Create a Pydantic model to accept input:

In [5]:
class TextGenerationInput(BaseModel):
    text: str
    model_name: str = "gpt2-large"

---
## Define the FastAPI endpoint for text generation:

In [6]:
@app.post("/generate")
async def generate_text(input_data: TextGenerationInput):
    model_name = input_data.model_name
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    input_text = input_data.text
    input_tokens = tokenizer.encode(input_text, return_tensors="pt")
    output_tokens = model.generate(input_tokens)
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    # Log interaction to Redis
    interaction_data = {
        "input": input_text,
        "output": output_text,
        "model_name": model_name,
    }
    redis_client.set("interaction", json.dumps(interaction_data))

    # Publish event to Redis Pub/Sub
    redis_client.publish("events", json.dumps(interaction_data))

    return {"generated_text": output_text}


---
## Set up WebSocket for real-time communication:

In [7]:
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()

    while True:
        data = await websocket.receive_text()
        input_data = TextGenerationInput(**json.loads(data))
        model_name = input_data.model_name
        model = GPT2LMHeadModel.from_pretrained(model_name)
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)

        input_text = input_data.text
        input_tokens = tokenizer.encode(input_text, return_tensors="pt")
        output_tokens = model.generate(input_tokens)
        output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

        # Log interaction to Redis
        interaction_data = {
            "input": input_text,
            "output": output_text,
            "model_name": model_name,
        }
        redis_client.set("interaction", json.dumps(interaction_data))

        # Publish event to Redis Pub/Sub
        redis_client.publish("events", json.dumps(interaction_data))

        # Send generated text via WebSocket
        await websocket.send_text(output_text)


> Note that the pyngrok package is optional and only needed if you want to expose your API to the internet using ngrok. To install the pyngrok package, run:

In [8]:
#pip install pyngrok
# pyngrok :
try:
    import pyngrok                      
    print('pyngrok: already installed')
except ImportError:
  !python -m pip install -q pyngrok
  print('Installed pyngrok')

pyngrok: already installed


---
## Run the FastAPI application in the Jupyter Notebook:

In [9]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

# Set up ngrok for external access (optional)
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)


t=2023-04-03T14:03:26-0700 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040
INFO:     Started server process [42316]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Public URL: http://4dbe-69-172-154-170.ngrok.io


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\wesle\anaconda3\envs\ViretualWorkspace\lib\site-packages\uvicorn\server.py:151 in       │
│ startup                                                                                          │
│                                                                                                  │
│   148 │   │   else:                                                                              │
│   149 │   │   │   # Standard case. Create a socket from a host/port pair.                        │
│   150 │   │   │   try:                                                                           │
│ ❱ 151 │   │   │   │   server = await loop.create_server(                                         │
│   152 │   │   │   │   │   create_protocol,                                                       │
│   153 │   │   │   │   │   host=config.host,                                                      │
│   154 │   │   │   │   │   port=config.port,                                                      │
│                                                                                                  │
│ C:\Users\wesle\anaconda3\envs\ViretualWorkspace\lib\asyncio\base_events.py:1506 in create_server │
│                                                                                                  │
│   1503 │   │   │   │   │   try:                                                                  │
│   1504 │   │   │   │   │   │   sock.bind(sa)                                                     │
│   1505 │   │   │   │   │   except OSError as err:                                                │
│ ❱ 1506 │   │   │   │   │   │   raise OSError(err.errno, 'error while attempting '                │
│   1507 │   │   │   │   │   │   │   │   │     'to bind on address %r: %s'                         │
│   1508 │   │   │   │   │   │   │   │   │     % (sa, err.strerror.lower())) from None             │
│   1509 │   │   │   │   completed = True                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket 
address (protocol/network address/port) is normally permitted

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\wesle\anaconda3\envs\ViretualWorkspace\lib\site-packages\IPython\core\interactiveshell. │
│ py:3433 in run_code                                                                              │
│                                                                                                  │
│   3430 │   │   │   │   if async_:                                                                │
│   3431 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3432 │   │   │   │   else:                                                                     │
│ ❱ 3433 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3434 │   │   │   finally:                                                                      │
│   3435 │   │   │   │   # Reset our crash handler in place                                        │
│   3436 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ C:\Users\wesle\AppData\Local\Temp\ipykernel_42316\3551604787.py:12 in <module>                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\we

---
## Test API with a gardio interface


In [ ]:
# # pip install gradio

# # gradio :
# try:
#     import gradio                      
#     print('gradio: already installed')
# except ImportError:
#   !python -m pip install -q gradio
#   print('Installed gradio')


In [ ]:
# import gradio as gr

# def generate_text(input_text):
#     import requests
#     response = requests.post("http://localhost:8000/generate", json={"text": input_text})
#     return response.json()["generated_text"]

# iface = gr.Interface(fn=generate_text, inputs="text", outputs="text", title="GPT-2 Text Generation")
# iface.launch()
